# 📙 LangChain Expression Language (LCEL)

---

## 📌 RunnablePassthrough

### 🎫 RunnablePassthrough 란?

- 입력을 아무 변경 없이 그대로 넘기는 블록 (데이터 전달 역할)

- 용도 : 자리 맡기용, 연결용, 디버깅용, 공백 처리용

- 일반적으로 `RunnableParallel` 과 함께 활용됩니다.

---

## 📌 RunnableParallel

### 🔃 RunnableParallel 란?

- 여러 개의 Runnable을 동시에 실행해주는 블록

- 한 개의 입력을 여러 개로 받고, 각각의 대한 결과물을 여러 개의 결과로 반환

---

### ✅ 검색기 사용에서 RunnablePassthrough를 사용하는 사례 살펴보기

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 문서
docs = [
    "문기는 랭체인이 좋습니다.",
    "민기는 랭체인이 싫습니다.",
    "문기의 직업은 엔지니어입니다.",
    "민기의 직업은 gang입니다."
]
vectorstore = FAISS.from_texts(docs, embedding=OpenAIEmbeddings())

# 벡터 저장소를 검색기로 사용
retriever = vectorstore.as_retriever()

# 템플릿 정의
template = """
Answer the question based only on the following context: {context}
Question: {question}
"""

# 템플릿으로부터 채팅 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문서를 포맷팅하는 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

# 검색 체인 구성
retrieval_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

### 실행 및 결과 확인해보기

In [8]:
retrieval_chain.invoke("문기의 직업은 무엇입니까?")

'문기의 직업은 엔지니어입니다.'

In [9]:
retrieval_chain.invoke("민기는 뭐하는 놈이야?")

'민기는 gang입니다.'

In [14]:
retrieval_chain.invoke("RunnablePassthrough은 무엇인가요?")

'주어진 맥락에는 "RunnablePassthrough"에 대한 정보가 포함되어 있지 않습니다. 따라서 답변할 수 없습니다.'

### 정리하기

☑️ "context" : retriever + format_docs 실행 결과

☑️ "question" : 입력값 그대로 들어가야 함

☑️ RunnablePassthrough() : 질문을 아무 수정 없이 그대로 다음 단계에 전달 목적용

☑️ RunnablePassthrough는 필수 인가?\
-> "question" 키를 처리하기 위해 필수 사항임❕\
-> 해당 클래스일 필요는 없지만 이를 대신할 무언가는 필수로 존재해야만 한다.

---

### ✅ 같은 질문에 서로 다른 프롬프트를 병렬로 적용해서 RunnableParallel 알아보기

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
model = ChatOpenAI()

# 프롬프트1 체인 정의
capital_chain = (
    ChatPromptTemplate.from_template("{country} 의 수도는 어디입니까?")
    | model
    | StrOutputParser()
)

# 프롬프트2 체인 정의
area_chain = (
    ChatPromptTemplate.from_template("{country} 의 면적은 얼마입니까?")
    | model
    | StrOutputParser()
)

# 병렬 실행 용 RunnableParallel 객체 생성
map_chain = RunnableParallel(capital=capital_chain, area=area_chain)

### 실행하기

In [6]:
# 질문하기
map_chain.invoke({"country": "대한민국"})

{'capital': '서울입니다.', 'area': '대한민국의 면적은 약 100,210 제곱 킬로미터입니다.'}

---

### ✅ RunnablePassthrough와 RunnableParallel 함께 알아보기

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 문서
docs = [
    "대한민국의 기후는 사계절이 뚜렷하다.",
    "대한민국은 겨울에 눈이 오고 여름엔 장마가 있다.",
    "대한민국의 수도는 서울이다.",
]
vectorstore = FAISS.from_texts(docs, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

# LLM & 파서
model = ChatOpenAI(model_name="gpt-4o-mini")
parser = StrOutputParser()

# format_docs 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

retriever_chain = retriever | format_docs

# 그대로 질문하는 체인 (Passthrough)
chain_direct = (
    {"question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template("질문: {question}\n정확히 대답해줘.")
    | model
    | parser
)

# RAG 체인 (문서 기반)
chain_rag = (
    {"context": retriever_chain, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(
        "다음 문서를 참고하여 질문에 답하세요.\n문서: {context}\n질문: {question}"
    )
    | model
    | parser
)

# 병렬로 실행 (RunnableParallel)
parallel_chain = RunnableParallel({
    "LLM 단독 답변": chain_direct,
    "LLM & RAG 답변": chain_rag
})

### 실행하기

In [13]:
result = parallel_chain.invoke("대한민국의 기후가 궁금해")

print("\n[LLM 단독 답변]\n")
print(result["LLM 단독 답변"])

print("\n[LLM & RAG 답변]\n")
print(result["LLM & RAG 답변"])


[LLM 단독 답변]

대한민국의 기후는 대체로 온대몬순 기후로 분류됩니다. 사계절이 뚜렷하게 나타나는 것이 특징이며, 각 계절의 기온과 강수량에 따라 다양한 날씨 패턴을 보입니다.

1. **봄(3월~5월)**: 맑고 따뜻한 날씨가 많으며, 꽃이 만개하는 시기입니다. 평균 기온은 약 10도에서 20도 사이입니다. 그러나 봄철에는 황사와 미세먼지가 발생할 수 있습니다.

2. **여름(6월~8월)**: 덥고 습한 날씨가 특징이며, 평균 기온은 25도에서 35도 사이입니다. 이 시기에는 장마가 시작되어 많은 비가 내리며, 습도가 높아져 불쾌지수가 올라갑니다.

3. **가을(9월~11월)**: 맑고 선선한 날씨로, 단풍이 아름답게 물드는 시기입니다. 평균 기온은 약 10도에서 20도 사이로, 쾌적한 날씨가 지속됩니다.

4. **겨울(12월~2월)**: 춥고 건조한 날씨가 이어지며, 평균 기온은 0도에서 -10도 사이입니다. 특히, 북부지역은 눈이 많이 오고, 남부지역은 상대적으로 온난한 겨울을 경험합니다.

전반적으로 대한민국은 사계절의 변화가 뚜렷하여 각 계절마다 다양한 자연 경관과 기후 특성을 즐길 수 있습니다. 또한, 지역에 따라 기후 차이가 있어, 서울과 부산, 제주도 등에서의 기온과 강수량은 다르게 나타날 수 있습니다.

[LLM & RAG 답변]

대한민국의 기후는 사계절이 뚜렷합니다. 겨울에는 눈이 내리고, 여름에는 장마가 있습니다. 이러한 특징 덕분에 각 계절마다 다양한 날씨와 풍경을 경험할 수 있습니다.


---
---

## 📌 Runnable 구조(그래프) 검토

### 📈 Runnable 구조(그래프) 검토
- Runnable의 흐름을 이해하기 위한 그래프 그려보기

### 라이브러리 설치

In [ ]:
# 필요 라이브러리 설치
!pip install -qU faiss-cpu tiktoken

# 그래프 라이브러리 설치
!pip install -qU grandalf

### 일반적인 체인 구성 샘플

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    # 텍스트 데이터로부터 FAISS 벡터 저장소를 생성합니다.
    ["Teddy는 AI 엔지니어입니다.", "Teddy는 프로그래밍을 좋아합니다!"],
    embedding=OpenAIEmbeddings(),
)

# 벡터 저장소를 기반으로 retriever를 생성합니다.
retriever = vectorstore.as_retriever()

template = """다음 맥락만을 토대로 질문에 답하세요.:
{context}  

Question: {question}"""

# 템플릿을 기반으로 ChatPromptTemplate을 생성합니다.
prompt = ChatPromptTemplate.from_template(
    template
)

# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI(model="gpt-4o-mini")

# chain 을 생성합니다.
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

### Runnable의 그래프 얻기

In [20]:
# 체인의 그래프에서 노드 가져오기
chain.get_graph().nodes

{'9e99e544dd5f46d39b9827e4f250790b': Node(id='9e99e544dd5f46d39b9827e4f250790b', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>, metadata=None),
 '1961b2b62b8d496f86fb81ffb2d0029e': Node(id='1961b2b62b8d496f86fb81ffb2d0029e', name='Parallel<context,question>Output', data=<class 'langchain_core.utils.pydantic.RunnableParallel<context,question>Output'>, metadata=None),
 '2b3e089446224bdda73e584caed61770': Node(id='2b3e089446224bdda73e584caed61770', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1694c8690>, search_kwargs={}), metadata=None),
 'dd95ff52bf2947749987f4c8ad52350e': Node(id='dd95ff52bf2947749987f4c8ad52350e', name='Passthrough', data=RunnablePassthrough(), metadata=None),
 '9f1ac3dec1f74dcd9a7a92d1dd28ffe5': Node(id='9f1ac3dec1f74dcd9a7a92d1dd28ffe5', name='ChatPromptTemplate', data=ChatP

- 노드(Node): 각각의 처리 단계 (예: retriever, prompt, model 등)

In [ ]:
# 체인의 그래프에서 엣지를 가져오기
chain.get_graph().edges

[Edge(source='c525746f8cf2494eb90f2c08097eded9', target='5e45775b2cb444c0a674d3d5004a1760', data=None, conditional=False),
 Edge(source='5e45775b2cb444c0a674d3d5004a1760', target='863efcd269944b3486ace0c82316e1f1', data=None, conditional=False),
 Edge(source='c525746f8cf2494eb90f2c08097eded9', target='d50f4826cfc74a28bdc8e3aaa3e31292', data=None, conditional=False),
 Edge(source='d50f4826cfc74a28bdc8e3aaa3e31292', target='863efcd269944b3486ace0c82316e1f1', data=None, conditional=False),
 Edge(source='863efcd269944b3486ace0c82316e1f1', target='46cae50d6a4e4934a1440d4afc4be280', data=None, conditional=False),
 Edge(source='46cae50d6a4e4934a1440d4afc4be280', target='f667874687d14d7fbe43aa9ffe4812da', data=None, conditional=False),
 Edge(source='ed2d1a69a5634b40b8f5e711d7004b53', target='6270c2309d4b4d4390a2cbc297860425', data=None, conditional=False),
 Edge(source='f667874687d14d7fbe43aa9ffe4812da', target='ed2d1a69a5634b40b8f5e711d7004b53', data=None, conditional=False)]

- 엣지(Edge): 그 처리 단계들 사이를 연결하는 “화살표”나 “선”

### 그래프 출력하기

In [ ]:
# 체인의 그래프를 ASCII 형식으로 출력
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

### ☑️ 정리하기

1.	ParallelInput: 체인의 맨 앞에서 "context"와 "question" 두 개의 key에 대해 동시에 처리 준비

2.	VectorStoreRetriever: "context" 값 생성을 위해 retriever 실행 (벡터 검색)

3.	Passthrough: "question"은 입력값 그대로 통과

4.	ParallelOutput: "context" + "question" → 딕셔너리로 묶어서 다음 단계로 넘김

5.	ChatPromptTemplate: {context}\nQuestion: {question} 형태의 프롬프트 생성

6.	ChatOpenAI: 프롬프트를 LLM(GPT-4o-mini)에 전달 → 응답 생성

7.	StrOutputParser: 모델 응답을 파싱해서 문자열로 변환

8.	StrOutputParserOutput: 최종 출력


---


----

----

## 📌 동적 속성 지정 (configurable_fields, configurable_alternatives)

### ⛓️‍💥 런타임에 체인 내부 구성하기

- `configurable_fields` 시스템의 설정 값을 정의하는 필드를 의미합니다.

- `configurable_alternatives` 런타임에 설정할 수 있는 Runnable 대안을 구성합니다.

---

### configurable_fields

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
print(prompt.configurable_fields())

default=ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}'), additional_kwargs={})]) fields={}


-> configurable_fields의 의해서 해당 템플릿은 교체 가능한 필드로 지정됨

### 언제 쓰이는가 ⁉️

- 런타임에 프롬프트를 교체하는 경우

- 동적으로 모델을 변경하거나, 파라미터 값을 실험하는 경우

- 설정 값을 .with_config() 등으로 바꿀 경우

- 서버, UI 등에서 외부 설정 주입해야하는 경우

### 런타임에 프롬프트를 교체하는 경우

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 기본 프롬프트
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

# 기본 체인 구성
chain = prompt | ChatOpenAI()

# 프롬프트를 교체하고 싶다면?
new_prompt = ChatPromptTemplate.from_template("Explain {topic} to a 5-year-old")
chain = chain.with_config(configurable={"prompt": new_prompt})

In [4]:
print(chain.configurable_fields())

default=RunnableBinding(bound=ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12215d550>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x120c5a150>, root_client=<openai.OpenAI object at 0x121287210>, root_async_client=<openai.AsyncOpenAI object at 0x1224c7d90>, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={}, config={'configurable': {'prompt': ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Explain {topic} to a 5-year-old'), additional_kwargs={})]

---

### 동적으로 모델을 변경하는 경우

In [5]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

model.invoke("대한민국의 수도는 어디야?").__dict__

{'content': '대한민국의 수도는 서울입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 8,
   'prompt_tokens': 15,
   'total_tokens': 23,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-2024-08-06',
  'system_fingerprint': 'fp_a288987b44',
  'id': 'chatcmpl-Bp6z13FM6gROnpOXHaNnflloxxKIt',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-bff8624b-0038-4fa1-93da-5fd01f367787-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 8,
  'total_tokens': 23,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

`configurable_fields` 메서드를 사용하여 `model_name` 속성을 동적 구성 가능한 필드로 지정합니다.

In [15]:
model = ChatOpenAI(temperature=0).configurable_fields(
    model_name=ConfigurableField(  # model_name 은 원래 ChatOpenAI 의 필드입니다.
        id="gpt_version",  # model_name 의 id 를 설정합니다.
        name="Version of GPT",  # model_name 의 이름을 설정합니다.
    )
)

`model.invoke()` 호출시 `config={"configurable": {"키": "값"}}` 형식으로 동적 지정할 수 있습니다.

In [16]:
model.invoke(
    "대한민국의 수도는 어디야?",
    # gpt_version 을 gpt-3.5-turbo 로 설정합니다.
    config={"configurable": {"gpt_version": "gpt-3.5-turbo"}},
).__dict__

{'content': '대한민국의 수도는 서울이다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 16,
   'prompt_tokens': 22,
   'total_tokens': 38,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'id': 'chatcmpl-Bp7ztHPzsqMM7XDZSXRJderAhtTz7',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-4f664bb5-2f59-4d5e-ab58-dbd682f7729f-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 22,
  'output_tokens': 16,
  'total_tokens': 38,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [8]:
model.invoke(
    # gpt_version 을 gpt-4o-mini 로 설정합니다.
    "대한민국의 수도는 어디야?",
    config={"configurable": {"gpt_version": "gpt-4o-mini"}},
).__dict__

{'content': '대한민국의 수도는 서울입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 8,
   'prompt_tokens': 15,
   'total_tokens': 23,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_34a54ae93c',
  'id': 'chatcmpl-Bp74KPMOKmYd5dmx5ySfMnRLKoBa4',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-d7bb556f-53d2-4705-a62e-a03716affe25-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 8,
  'total_tokens': 23,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

---

### 설정값을 .with_config() 바꿀 경우

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 원래 체인
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
llm = ChatOpenAI(model_name="gpt-4o-mini")
chain = prompt | llm

# 프롬프트를 더 친절하게 바꾸고 싶을 때
new_prompt = ChatPromptTemplate.from_template("Can you kindly tell a joke about {topic}?")

# 프롬프트만 교체
reconfigured_chain = chain.with_config(configurable={"prompt": new_prompt})

# 실행
print(reconfigured_chain.invoke({"topic": "bananas"}))

content="Why did the banana go to the doctor? \n\nBecause it wasn't peeling very well! 🍌" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 13, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpP1kfGNEVptGeXMzIcEDgdM8jNJv', 'finish_reason': 'stop', 'logprobs': None} id='run--e395786e-c94c-4163-a073-e9122c44f5bf-0' usage_metadata={'input_tokens': 13, 'output_tokens': 19, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


---
---

### Configuable Alternatives: Runnable 객체 자체의 대안 설정

- `ChatAnthropic` 의 구성 가능한 언어 모델은 다양한 작업과 컨텍스트에 적용할 수 있는 유연성을 제공합니다.

- 동적으로 설정(Config) 값을 변경하기 위하여 모델에 설정하는 파라미터를 ConfigurableField 객체로 설정합니다.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatAnthropic(
    temperature=0, model="claude-3-5-sonnet-20240620"
).configurable_alternatives(

    ConfigurableField(id="llm"),

    default_key="anthropic",

    openai=ChatOpenAI(model="gpt-4o-mini"),
    gpt4o=ChatOpenAI(model="gpt-4o"),

)
prompt = PromptTemplate.from_template("{topic} 에 대해 간단히 설명해주세요.")
chain = prompt | llm


In [ ]:
# Anthropic을 기본으로 호출합니다.
chain.invoke({"topic": "뉴진스"}).__dict__

{'content': '뉴진스(NewJeans)는 2022년 7월 22일에 데뷔한 대한민국의 5인조 걸그룹입니다. 주요 특징은 다음과 같습니다:\n\n1. 소속사: ADOR (HYBE Labels의 자회사)\n\n2. 멤버: 민지, 하니, 다니엘, 해린, 혜인\n\n3. 데뷔곡: "Attention", "Hype Boy", "Cookie"\n\n4. 특징:\n   - 10대 멤버들로 구성된 신선한 이미지\n   - Y2K, 90년대 감성을 현대적으로 재해석한 음악과 스타일\n   - 독특한 마케팅 전략 (사전 홍보 없이 갑작스러운 데뷔)\n\n5. 주요 성과:\n   - 데뷔 앨범 \'뉴 진스(New Jeans)\'로 빌보드 200 차트 진입\n   - 여러 음원 차트 1위 기록\n   - 각종 신인상 수상\n\n뉴진스는 데뷔 이후 빠르게 인기를 얻으며 4세대 K-pop을 대표하는 그룹으로 자리매김하고 있습니다.', 'additional_kwargs': {}, 'response_metadata': {'id': 'msg_01PJfWdBpgp2FbWp1t3HfQP4',  'model': 'claude-3-5-sonnet-20240620',  'stop_reason': 'end_turn',  'stop_sequence': None,  'usage': {'input_tokens': 30, 'output_tokens': 390}}, 'type': 'ai', 'name': None, 'id': 'run-68b3570d-a0d4-4074-9d69-cbf40b3caf8b-0', 'example': False, 'tool_calls': [], 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 30,  'output_tokens': 390,  'total_tokens': 420}}

-> ChatAnthropic 모델을 사용하기 위하여 API KEY를 발급받아 설정해야합니다.

---
---

## 📌 RunnableWithMessageHistory

### 📝 RunnableWithMessageHistory란?

- RunnableWithMessageHistory는 LangChain 체인에 “채팅 메시지 히스토리”를 연결해주는 래퍼

- 사용자의 대화를 계속 기억하면서 “대화형 챗봇”처럼 동작하게 만들고 싶을 때 사용

### 실제 활용 예시

- 대화형 챗봇 개발: 사용자와의 대화 내역을 기반으로 챗봇의 응답을 조정할 수 있습니다.

- 복잡한 데이터 처리: 데이터 처리 과정에서 이전 단계의 결과를 참조하여 다음 단계의 로직을 결정할 수 있습니다.

- 상태 관리가 필요한 애플리케이션: 사용자의 이전 선택을 기억하고 그에 따라 다음 화면이나 정보를 제공할 수 있습니다.

### 휘발성 대화기록: 인메모리(In-Memory)

사용자와 주고받은 메시지를 메모리에만 잠깐 저장하고,

프로그램이 종료되면 기억이 사라지는 구조

In [48]:
from langchain_core.chat_history import InMemoryChatMessageHistory

# 대화 히스토리 객체 생성 (휘발성)
history = InMemoryChatMessageHistory()

# 사용자 → 시스템 메시지 저장
history.add_user_message("안녕?")
history.add_ai_message("안녕하세요! 무엇을 도와드릴까요?")

# 출력해보기
for msg in history.messages:
    print(f"[{msg.type}] {msg.content}")

[human] 안녕?
[ai] 안녕하세요! 무엇을 도와드릴까요?


### 휘발성 실사용 예시

In [3]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import InMemoryChatMessageHistory

# 프롬프트 구성
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}"),
])

# 체인
chain = prompt | ChatOpenAI(model_name="gpt-4o-mini") | StrOutputParser()

# 세션별 메모리 저장소
store = {}

# get_session_history 함수 수정
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메시지 히스토리를 붙인 체인 구성
chat_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [4]:
session_id = "user-1234"

print(chat_chain.invoke(
    {"input": "한국의 수도는 어디야?"},
    config={"configurable": {"session_id": session_id}}
))

print(chat_chain.invoke(
    {"input": "그 도시의 날씨는 어때?"},
    config={"configurable": {"session_id": session_id}}
))

한국의 수도는 서울입니다.
어떤 도시의 날씨를 알고 싶으신가요? 도시의 이름을 알려주시면 그 지역의 날씨에 대한 정보를 제공해 드릴 수 있습니다.


-> 출력마다 InMemory에 의해 대화 기록이 휘발되어 그 도시라고 질문하게 되면 출력하지 못하게 된다.

---

### ✅ 메시지 기록(메모리) 추가하기

### 🛑 RedisChatMessageHistory

- Redis를 통한 대화 기록 영구 저장

In [59]:
%pip install -qU redis

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Redis 세션 히스토리 불러오기 함수
def get_history(session_id: str):
    return RedisChatMessageHistory(
        session_id=session_id,
        url="redis://223.130.143.231:6379",  # 🔧 본인 환경에 맞게 조정 (cloud이면 URL로)
        ttl=None  # None이면 무제한 저장
    )

# 프롬프트 구성 — 반드시 {history} 포함!
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("placeholder", "{history}"),
    ("human", "{input}"),
])

# 체인 정의
base_chain = prompt | ChatOpenAI(model_name="gpt-4o-mini") | StrOutputParser()

# 메시지 히스토리 적용
chat_chain = RunnableWithMessageHistory(
    base_chain,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [ ]:
session_id = "user-5678"  # 고정된 사용자 ID

# 1. 첫 번째 질문
print(chat_chain.invoke(
    {"input": "한국의 수도는 어디야?"},
    config={"configurable": {"session_id": session_id}}
))

# 2. 이어지는 문맥 질문 (다시 실행해도 Redis가 기억)
print(chat_chain.invoke(
    {"input": "그 도시의 날씨는 어때?"},
    config={"configurable": {"session_id": session_id}}
))

한국의 수도는 서울입니다.
현재 서울의 날씨는 실시간 정보가 필요하기 때문에, 구체적인 날씨 정보를 제공해 드릴 수는 없습니다. 하지만 일반적으로 서울의 날씨는 사계절이 뚜렷하며, 여름은 덥고 습하며, 겨울은 춥고 건조합니다. 가을과 봄은 비교적 온화한 날씨를 보입니다. 정확한 날씨 정보를 알고 싶다면 날씨 예보 웹사이트나 모바일 앱을 확인하는 것이 좋습니다.


---

### 🛑 RunnableWithMessageHistory

- 애플리케이션의 상태를 유지하고, 사용자 경험을 향상시키며, 더 정교한 응답 메커니즘을 구현할 수 있게 해주는 강력한 도구

In [1]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}  # 세션 기록을 저장할 딕셔너리


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        runnable,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="input",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)


NameError: name 'runnable' is not defined

In [90]:
# 그냥 RunnableWithMessageHistory만 쓰기
chat_chain = RunnableWithMessageHistory(
    chain,
    get_session_history=lambda session_id: ...,  # history return
    input_messages_key="input",
    history_messages_key="history",
)

In [96]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
).__dict__

abc123


{'content': 'Ratio of the adjacent side length to the hypotenuse length in a right triangle.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 17,
   'prompt_tokens': 160,
   'total_tokens': 177,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'id': 'chatcmpl-BpBmf29kO2x9yMy6ItNq3ClkztIeC',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-9e34f702-7d71-4a5a-b960-aa7aa57af9d7-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 160,
  'output_tokens': 17,
  'total_tokens': 177,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [97]:
# 메시지 기록을 포함하여 호출합니다.
with_message_history.invoke(
    # 능력과 입력을 설정합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요."},
    # 설정 옵션을 지정합니다.
    config={"configurable": {"session_id": "abc123"}},
).__dict__

abc123


{'content': '직각 삼각형에서 인접한 변의 길이를 빗변의 길이로 나눈 비율입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 41,
   'prompt_tokens': 202,
   'total_tokens': 243,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'id': 'chatcmpl-BpBmmVi7wssMOWjPhIrVOFDthCK2e',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-bb95f1e5-d58b-4196-a5e5-008b11f03342-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 202,
  'output_tokens': 41,
  'total_tokens': 243,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

---

추가 실험

### 대화 히스토리 시각화

In [ ]:
import redis, json

r = redis.Redis(host="223.130.143.231", port=6379, decode_responses=True)
# Redis에 저장된 대화 히스토리 키
key = "message_store:user-5678"

# Redis에서 대화 히스토리 가져오기
messages = r.lrange(key, 0, -1)

# 필요한 필드 출력시키기
print("대화 히스토리\n")
for raw in reversed(messages):
    msg = json.loads(raw)
    role = msg["type"]
    content = msg["data"]["content"]
    role = "사용자" if role == "human" else "GPT"
    print(f"{role}: {content}\n")

대화 히스토리

사용자: 한국의 수도는 어디야?

GPT: 한국의 수도는 서울입니다.

사용자: 그 도시의 날씨는 어때?

GPT: 현재 서울의 날씨는 실시간 정보가 필요하기 때문에, 구체적인 날씨 정보를 제공해 드릴 수는 없습니다. 하지만 일반적으로 서울의 날씨는 사계절이 뚜렷하며, 여름은 덥고 습하며, 겨울은 춥고 건조합니다. 가을과 봄은 비교적 온화한 날씨를 보입니다. 정확한 날씨 정보를 알고 싶다면 날씨 예보 웹사이트나 모바일 앱을 확인하는 것이 좋습니다.



### 위의 히스토리 내용을 FAISS 저장소에 저장한 후 검색기로 AI 답변 구하기

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 벡터화(임베딩)된 문서들을 가져와서 FAISS 벡터 저장소를 생성
docs = [json.loads(m)["data"]["content"] for m in messages]
vectorstore = FAISS.from_texts(docs, embedding=OpenAIEmbeddings())

# 벡터저장소를 검색기로 사용
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# AI가 답변할 템플릿 형식 정의
prompt = ChatPromptTemplate.from_template("""
주어진 문서를 바탕으로 질문에 대답하세요.

문서:
{context}

질문:
{question}
""")

# 전체 RAG 체인 구성
rag_chain = (
    # 딕셔너리를 처리하기 위한 람다 함수
    {"context": lambda x: retriever.invoke(x["question"]), "question": lambda x: x["question"]}
    | prompt
    | ChatOpenAI(model_name="gpt-4o-mini")
    | StrOutputParser()
)

In [15]:
# 질문 실행
print(rag_chain.invoke({"question": "그 도시의 날씨는 어때?"}))

현재 서울의 날씨에 대한 구체적인 정보를 제공할 수는 없지만, 일반적으로 서울의 날씨는 사계절이 뚜렷합니다. 여름은 덥고 습하며, 겨울은 춥고 건조합니다. 가을과 봄은 비교적 온화한 날씨를 보입니다. 정확한 날씨 정보를 원하신다면 날씨 예보 웹사이트나 모바일 앱을 확인하는 것이 좋습니다.


### -> Redis에 사용자 고정 세션값안에서 자유롭게 AI와 대화하고, 해당 세션값에 저장된 대화 내용들을 바로 바로 FAISS나 Chroma 저장소에 임베딩 시켜 저장하면, 나만의 챗봇을 구현해 볼 수 있다.

---

## @chain 데코레이터란

- 우선 Python의 데코레이터란? 함수나 메서드의 기능을 확장하거나 변경하는 역할을 합니다.

### 일반적인 함수 호출 코드

In [23]:
def my_decorator(func):
    def wrapper():
        print("함수 실행 전...")
        func()
        print("함수 실행 후!!")
    return wrapper

def hello():
    print("안녕하세요")

decorated_hello = my_decorator(hello)
decorated_hello()

함수 실행 전...
안녕하세요
함수 실행 후!!


### @데코레이터 형식 코드

In [24]:
def my_decorator(func):
    def wrapper():
        print("함수 실행 전...")
        func()
        print("함수 실행 후!!")
    return wrapper

@my_decorator
def say_hello():
    print("안녕하세요")

say_hello()

함수 실행 전...
안녕하세요
함수 실행 후!!


- my_decorator 함수는 say_hello 함수를 감싸고 있는 형태라고 볼 수 있습니다.

- `@my_decorator` 는 say_hello = my_decorator(say_hello)와 동일한 표현입니다.

In [25]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("함수 호출 전...")
        result = func(*args, **kwargs)
        print("함수 호출 후!!")
        return result
    return wrapper

@my_decorator
def add(a, b):
    print(f"계산 중: {a} + {b}")
    return a + b

result = add(3, 5)
print("결과:", result)

함수 호출 전...
계산 중: 3 + 5
함수 호출 후!!
결과: 8


1. @my_decorator를 보고, add를 my_decorator(add)로 치환한다.

2. my_decorator함수는 내부 wrapper함수를 수행하고 wrapper를 반환한다.

### @chain 데코레이터란?

- 함수를 하나의 Runnable 체인으로 자동 변환해주는 데코레이터

-> 쉽게 말해서 @chain을 함수 위에 붙이면, 그 함수는 자동으로 체인처럼 .invoke() .stream() .batch()등을 사용할 수 있습니다.

| 간단한 기본 사용법 예시

In [39]:
from langchain_core.runnables import chain

@chain
def weather_chain(input: str) -> str:
    return f"오늘의 {input} 날씨는 어떤가요?"

- 이제 weather_chain은 LangChain의 Runnable처럼 사용할 수 있다.

In [40]:
print(weather_chain.invoke("부산"))

오늘의 부산 날씨는 어떤가요?


|  LLM과 함께 사용될 경우 예시

In [36]:
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini")
parser = StrOutputParser()

@chain
def city_chain(input: str) -> str:
    prompt = ChatPromptTemplate.from_template("{input}의 도시는 어떤거 같아?")
    return prompt | llm | parser

- 마찬가지로 city_chain함수는 @chain데코레이터 처리되어 Runnable처럼 사용할 수 있다.

In [37]:
print(city_chain.invoke("부산"))

부산은 한국에서 두 번째로 큰 도시로, 아름다운 해변과 독특한 문화, 맛있는 음식으로 유명합니다. 해운대, 광안리와 같은 해변은 특히 여름철에 많은 관광객들이 찾는 인기 명소입니다. 또한, 부산 국제영화제와 같은 다양한 문화 행사도 열리며, 이곳의 활기찬 분위기를 더해줍니다.

부산은 또한 항구 도시로, 국제적인 무역과 물류의 중심지 역할을 하고 있습니다. 이로 인해 다양한 문화가 어우러지는 독특한 매력을 지니고 있습니다. 그리고 지역 특산물인 밀면, 씨앗호떡, 생선회 등은 음식으로도 많은 사랑을 받고 있습니다.

부산은 자연경관도 뛰어나해 동백섬, 태종대, 금정산 등의 관광명소가 있어 다양한 야외 활동을 즐길 수 있습니다. 이렇듯 부산은 현대적인 도시의 매력과 자연의 아름다움이 조화를 이루는 특별한 곳입니다.


### @chain 데코레이터는 언제 혹은 어디에 쓰이는 가?

- .invoke() .stream() 등 Runnable 프로토콜 자동 적용되어 장점이 있다.

- 복잡한 체인을 직접 | 파이프로 연결하지 않고 함수로 정의하고 싶을 때 사용할 수 있다.

- 체인을 함수처럼 필요할 때만 사용될 수 있다.

| 교재에 내용대로 @chain 데코레이터로 Runnable 객체 사용

In [41]:
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

- prompt1 주어진 주제에 대한 짧은 설명 프롬프트
- prompt2 영어로 번역해달라는 요청 프롬프트

In [42]:
prompt1 = ChatPromptTemplate.from_template("{topic} 에 대해 짧게 한글로 설명해주세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{sentence} 를 emoji를 활용한 인스타그램 게시글로 만들어주세요."
)

- custom_chain 함수를 정의하고 데코레이터로 Runnable한 객체로 만듭니다.

In [43]:
@chain
def custom_chain(text):
    chain1 = prompt1 | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
    output1 = chain1.invoke({"topic": text})

    chain2 = prompt2 | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
    return chain2.invoke({"sentence": output1})

- custom_chain 함수를 호출합니다.

In [44]:
print(custom_chain.invoke("양자역학"))

🐾✨ 양자역학: 미세한 세계의 비밀 🔬⚛️

양자역학은 🦠 아원자 입자와 같은 작은 스케일의 동작을 설명하는 물리학의 한 분야입니다! 🔍 고전 물리학으로는 이해할 수 없는 신비로운 현상들을 다뤘죠. 

주요 개념으로는:
🌊 파동-입자 이중성
🔒 불확실성 원리
🔗 양자 얽힘

양자역학은 물질의 기본 구조와 상호작용을 이해하는 데 필수적이며, 🚀 반도체, 🌟 레이저, ☢️ 원자력 등 현대 기술의 기초를 다지고 있어요! 💡

#양자역학 #물리학 #과학 #파동입자이중성 #불확정성원리 #양자얽힘 #현대기술 #공부하는즐거움
